# Here is the plan:

* Scrape game names and id's using the `games` endpoint filtered by console
* Scrape individual game content for each game from previous scrape
    * Want the `description`, `genres`, `themes`, `producers`, `devs`, maybe even the `people` involved
* Scrape all user reviews for console
    * Want the `score`, `username`, and `content` of review
* Clean data:
    *  Drop games or users with too few reviews

In [1]:
from bs4 import BeautifulSoup
import requests
import json
import requests
import csv
import time

|Console | GUID |
|---:|---|
|Xbox One | 3045-145|
|PS4 | 3045-146 |

In [2]:
key = 'f05d9dbc286a3eb1f52355ea793a3e96c66a347f'
offset = 0

console_field_list = ['name', 'guid', 'company', 'release_date']
platform = '146'


console_url = f'https://www.giantbomb.com/api/platforms/?api_key={key}&format=json&sort=release_date:desc&limit=10&field_list={",".join(console_field_list)}'
headers = {'User-agent': 'LookAYoYo\'s open source recommender system'}


In [3]:
def log_init():
    with open('log.txt', 'w') as log:
            log.write('Beginning Log\n')
def log_write(text):
    with open('log.txt', 'a') as log:
            log.write(f'{text}\n')

# Game

In [4]:
def getURL(offset = 0):
    games_field_list = ['name', 'guid']
    games_url = f'https://www.giantbomb.com/api/games/?api_key={key}&format=json&offset={offset}&platforms={platform}&sort=number_of_user_reviews:desc&field_list={",".join(games_field_list)}'
    return games_url

def getJsonFromResponse(url):
    res = requests.get(url, headers= headers)
    soup = BeautifulSoup(res.content, "lxml")
    json_response = json.loads(soup.find('body').text)
    return json_response

def isDone(json_response):
    return json_response['number_of_page_results'] + json_response['offset'] > json_response['number_of_total_results']

def getAllGames(platform):
    log_init()
    start=time.time()
    flag = True
    offset = 0
    games_list = []
    while flag:
        url = getURL(offset)
        json_response = getJsonFromResponse(url)
        games_list.extend(json_response['results'])
        log_write(f'offset = {offset}')
        log_write(f'status = {json_response["error"]}')
        log_write(f'list_length = {len(games_list)}')
        log_write(f'runtime = {time.time() - start} seconds')
        if isDone(json_response):
            flag = False
        else:
            offset += 100
    return games_list

In [5]:
games_list = getAllGames(platform)

In [6]:
len(games_list)

2758

In [7]:
game = games_list[5]
guid = game['guid']
print(game['name'])

LocoRoco


In [8]:
with open("games_list.json", 'w+') as f:
    json.dump(games_list, f)

In [14]:
with open("games_list.json", 'r') as f:
    games_list = json.load(f)

In [17]:
games_list[1]

{'guid': '3030-2133', 'name': 'Call of Duty 4: Modern Warfare'}

# Platform

In [13]:
# res = requests.get(console_url, headers= headers)


# soup = BeautifulSoup(res.content, "lxml")

# json.loads(soup.find('body').text)

# Reviews

In [11]:
def getReviewURL(offset = 0):
    rev_field_list = ['name', 'guid', 'deck', 'description', 'reviewer', 'score', 'site_detail_url']
    reviews_url= f'https://www.giantbomb.com/api/user_reviews/?api_key={key}&format=json&offset={offset}&field_list={",".join(rev_field_list)}'    
    return reviews_url
def getAllReviews(offset = 0):

    log_init()
    start=time.time()
    flag = True
#     rev_list = []
    while flag:
        url = getReviewURL(offset)
        json_response = getJsonFromResponse(url)
#         rev_list.extend(json_response['results'])
        log_write(f'offset = {offset}')
        log_write(f'status = {json_response["error"]}')
        #log_write(f'list_length = {len(rev_list)}')
        log_write(f'runtime = {time.time() - start} seconds')
        with open(f'./review_data/reviews_list_{offset}.json', 'w') as f:
            json.dump(json_response['results'], f)
        if isDone(json_response) or (len(json_response['results']) == 0):
            flag = False
        else:
            offset += 100
            time.sleep(2)
#     return rev_list

In [12]:
rev_list = getAllReviews()

In [48]:
reviews_url= getReviewURL(2900)
res = requests.get(reviews_url, headers= headers)


soup = BeautifulSoup(res.content, "lxml")

json.loads(soup.find('body').text)

{'error': 'OK',
 'limit': 100,
 'number_of_page_results': 100,
 'number_of_total_results': 29249,
 'offset': 2900,
 'results': [{'deck': 'Online review for Vegas 2',
   'description': "If anyone has play Rainbowsix Vegas 1 you would know how hard it is to level up, but in Vegas 2 leveling up is as easy as spawn killing in halo. Vegas 2 offers the user a few new guns and some new maps.\xa0 But compared to Vegas 1, Vegas 2 is a huge let down online(well for me atleast).The problem for me on Vegas 2 online was that it wasnt any diffrent than number 1 execpt for the fact that leveling up was easy as pie. It seems that the only map people play is villa which can get boring after a while and kill house which is mostly used for achievement whores.\xa0 And when Ubisoft relized noone liked the new maps what did they do they realesed vegas 1's old maps in hope that people would start playing again. I dont mean to completely bash this game but if your looking for a challengeing online then i dont